In [ ]:
!pip install -r ../requirements.txt

DISCLAIMER: THIS CODE IS NOT MINE ⏰ BUT IT WORKS!

In [12]:
import os
import cv2
import albumentations as A
from glob import glob
from tqdm import tqdm
import shutil

# Paths (modify accordingly)
img_dir = '/content/5-cats-1/train/images'
label_dir = '/content/5-cats-1/train/labels'
aug_img_dir = '/content/5-cats-1/augTrain/images'
aug_label_dir = '/content/5-cats-1/augTrain/labels'

os.makedirs(aug_img_dir, exist_ok=True)
os.makedirs(aug_label_dir, exist_ok=True)

# Define augmentation pipeline
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=15, p=0.5),
    A.Blur(p=0.2),
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

# Helper function
def read_yolo_labels(label_path):
    boxes, class_labels = [], []
    with open(label_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            class_labels.append(int(parts[0]))
            boxes.append([float(x) for x in parts[1:]])
    return boxes, class_labels

def save_yolo_labels(save_path, boxes, class_labels):
    with open(save_path, 'w') as f:
        for cls, box in zip(class_labels, boxes):
            box_str = ' '.join(map(str, box))
            f.write(f"{cls} {box_str}\n")

# Augmentation Loop
image_paths = glob(os.path.join(img_dir, '*.jpg'))

for img_path in tqdm(image_paths):
    file_name = os.path.basename(img_path)
    label_path = os.path.join(label_dir, file_name.replace('.jpg', '.txt'))

    image = cv2.imread(img_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    boxes, class_labels = read_yolo_labels(label_path)

    # Apply augmentation
    augmented = transform(image=image, bboxes=boxes, class_labels=class_labels)

    aug_image = cv2.cvtColor(augmented['image'], cv2.COLOR_RGB2BGR)
    aug_boxes = augmented['bboxes']
    aug_class_labels = augmented['class_labels']

    # Skip images without bounding boxes after augmentation
    if len(aug_boxes) == 0:
        continue

    # Save augmented image and labels
    aug_file_name = f"aug_{file_name}"
    cv2.imwrite(os.path.join(aug_img_dir, aug_file_name), aug_image)
    save_yolo_labels(os.path.join(aug_label_dir, aug_file_name.replace('.jpg', '.txt')), aug_boxes, aug_class_labels)


ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
100%|██████████| 3/3 [00:00<00:00, 20.06it/s]


Apply pre-processing